# Client Graph
Notebook to manipulate a dask graph at the client stage.

## Example 1: Edit submit method

In [1]:
from dask.distributed import Client

# Create a Dask client
client = Client()

original_submit = client.submit

In [2]:
def modified_submit(*args, **kwargs):
    print("Entered Submit")
    print("Party if it works")
    return original_submit(*args, **kwargs)

In [3]:
client.submit = modified_submit

In [4]:
def neg(x):
    return -x


client.submit(neg, 3)

Entered Submit
Party if it works


<Future: pending, key: neg-6509467a9c66ce2389f8f71ccfe663b3>

In [5]:
client.shutdown()

2024-06-19 19:37:55,387 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:16903' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'neg-6509467a9c66ce2389f8f71ccfe663b3'} (stimulus_id='handle-worker-cleanup-1718822275.387008')


## Example 2: Edit collections_to_dsk method

In [6]:
from dask.distributed import Client
import dask
import dask.array as da

from datetime import datetime

client = Client()

# Save original collections_to_dsk method
original_collections_to_dsk = client.collections_to_dsk

In [7]:
def modified_collections_to_dsk(collections, *args, **kwargs):
    c = datetime.now()
    print("Inside myFunc. Time = " + c.strftime("%H:%M:%S"))
    print("Dask Graph Tasks")
    for collection in collections:
        print(f"Task Name: {collection.name}")
    print("-----------------\n")

    return original_collections_to_dsk(collections, *args, **kwargs)


client.collections_to_dsk = modified_collections_to_dsk
print("collections_to_dsk method has been replaced")

collections_to_dsk method has been replaced


In [8]:
x = da.ones((1000, 1000), chunks=(100, 100))
y = x + 1
z = y.mean()

In [9]:
result = client.compute(z)

Inside myFunc. Time = 19:38:18
Dask Graph Tasks
Task Name: mean_agg-aggregate-c50263a0ad47c330577b3058195ad928
-----------------



In [10]:
client.shutdown()

2024-06-19 19:38:31,549 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:21019' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'finalize-3daf4f11543bcdbd0d181b8d6c929af3'} (stimulus_id='handle-worker-cleanup-1718822311.5493789')


# Example 3: Replace a Task Graph

### Example 3.1: Keep client running
This will cause an issue as the client still has memory of the task and hash

In [11]:
from dask.distributed import Client
import dask
import dask.array as da
from dask.highlevelgraph import HighLevelGraph
from datetime import datetime

client = Client()

original_collections_to_dsk = client.collections_to_dsk

In [12]:
existing_functions = []


def add_to_three(number):
    return number + 3


def modified_collections_to_dsk(collections, *args, **kwargs):
    c = datetime.now()
    print("Inside myFunc. Time = " + c.strftime("%H:%M:%S"))

    print("Functions already processed")
    global existing_functions
    for funcs in existing_functions:
        print(f"{funcs}")

    print("Dask Graph Tasks")
    for collection in collections:
        print(f"Task Name: {collection.name}")
        print("Dask Graph")
        print(collection.dask)
        if collection.name not in existing_functions:
            print(f"Adding {collection.name} to existing_functions")
            existing_functions.append(collection.name)
        else:
            print(f"{collection.name} has already been processed")
            # TODO: Continue playing around with the dask graph so return is not a string
            # TODO: Use collection hash rather than collection name
            layers = {
                f"add_to_three-{(collection.name)}": {
                    (f"add_to_three-{(collection.name)}", 0): (add_to_three, 20)
                }
            }
            dependencies = {f"add_to_three-{(collection.name)}": set()}
            graph = HighLevelGraph(layers, dependencies)
            collection.dask = graph
            print("New Dask Graph")
            print(collection.dask)

    print("-----------------")

    return original_collections_to_dsk(collections, *args, **kwargs)


client.collections_to_dsk = modified_collections_to_dsk

In [13]:
x = da.ones((1000, 1000), chunks=(100, 100))
y = x + 1
z = y.mean()

In [14]:
result = client.compute(z)

Inside myFunc. Time = 19:40:19
Functions already processed
Dask Graph Tasks
Task Name: mean_agg-aggregate-c50263a0ad47c330577b3058195ad928
Dask Graph
HighLevelGraph with 7 layers.
 0. ones_like-c53a97836143a00162e3470fef213e1e
 1. add-098526cf93d1ac0d3a8744f59d0814fb
 2. mean_chunk-4e5168d025885dfbdfd76df3f8083cda
 3. mean_combine-partial-0832826c732652f3243668d740a0d5f3
 4. mean_combine-partial-f33215f12dcb169d48ded3405ae5b4a2
 5. mean_combine-partial-f380b3499d120118c907e62d760350cc
 6. mean_agg-aggregate-c50263a0ad47c330577b3058195ad928

Adding mean_agg-aggregate-c50263a0ad47c330577b3058195ad928 to existing_functions
-----------------


In [15]:
result

<Future: finished, type: numpy.float64, key: finalize-3daf4f11543bcdbd0d181b8d6c929af3>

In [16]:
result.result()

2.0

In [17]:
# Compute a second time
result = client.compute(z)

Inside myFunc. Time = 19:40:23
Functions already processed
mean_agg-aggregate-c50263a0ad47c330577b3058195ad928
Dask Graph Tasks
Task Name: mean_agg-aggregate-c50263a0ad47c330577b3058195ad928
Dask Graph
HighLevelGraph with 7 layers.
 0. ones_like-c53a97836143a00162e3470fef213e1e
 1. add-098526cf93d1ac0d3a8744f59d0814fb
 2. mean_chunk-4e5168d025885dfbdfd76df3f8083cda
 3. mean_combine-partial-0832826c732652f3243668d740a0d5f3
 4. mean_combine-partial-f33215f12dcb169d48ded3405ae5b4a2
 5. mean_combine-partial-f380b3499d120118c907e62d760350cc
 6. mean_agg-aggregate-c50263a0ad47c330577b3058195ad928

mean_agg-aggregate-c50263a0ad47c330577b3058195ad928 has already been processed
New Dask Graph
HighLevelGraph with 1 layers.
 0. add_to_three-mean_agg-aggregate-c50263a0ad47c330577b3058195ad928

-----------------


2024-06-19 19:40:23,945 - distributed.scheduler - WARNING - Detected different `run_spec` for key 'finalize-3daf4f11543bcdbd0d181b8d6c929af3' between two consecutive calls to `update_graph`. This can cause failures and deadlocks down the line. Please ensure unique key names. If you are using a standard dask collections, consider releasing all the data before resubmitting another computation. More details and help can be found at https://github.com/dask/dask/issues/9888. 
Debugging information
---------------------
old task state: memory
old run_spec: (<function finalize at 0x7f96080ec280>, ([('mean_agg-aggregate-c50263a0ad47c330577b3058195ad928',)],), {})
new run_spec: (<function finalize at 0x7f96080ec280>, ([('mean_agg-aggregate-c50263a0ad47c330577b3058195ad928',)],), {})
old token: ('tuple', [('ae65af090ad79ae2e434bc4c7eb750244a1a40bb', []), ('tuple', [('list', [('tuple', ['mean_agg-aggregate-c50263a0ad47c330577b3058195ad928'])])]), ('dict', [])])
new token: ('tuple', [('ae65af090ad

In [18]:
client.shutdown()

2024-06-19 19:40:46,904 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:13361' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'finalize-3daf4f11543bcdbd0d181b8d6c929af3'} (stimulus_id='handle-worker-cleanup-1718822446.9042947')


### Example 3.2: Edit HighLevelGraph in a new cluster

In [19]:
from dask.distributed import Client
import dask
import dask.array as da
from dask.highlevelgraph import HighLevelGraph
from datetime import datetime

In [20]:
client = Client()

original_collections_to_dsk = client.collections_to_dsk

In [21]:
existing_functions = ["mean_agg-aggregate-c50263a0ad47c330577b3058195ad928"]


def modified_collections_to_dsk(collections, *args, **kwargs):
    c = datetime.now()
    print("Inside myFunc. Time = " + c.strftime("%H:%M:%S"))

    print("Functions already processed")
    global existing_functions
    for funcs in existing_functions:
        print(f"{funcs}")

    print("Dask Graph Tasks")
    for collection in collections:
        print(f"Task Name: {collection.name}")
        print("Dask Graph")
        print(collection.dask)
        if collection.name not in existing_functions:
            print(f"Adding {collection.name} to existing_functions")
            existing_functions.append(collection.name)
        else:
            print(f"{collection.name} has already been processed")
            # TODO: Continue playing around with the dask graph so return is not a string
            # TODO: Use collection hash rather than collection name
            layers = {
                f"add_to_three-{(collection.name)}": {
                    (f"add_to_three-{(collection.name)}", 0): (add_to_three, 20)
                }
            }
            dependencies = {f"add_to_three-{(collection.name)}": set()}
            graph = HighLevelGraph(layers, dependencies)
            collection.dask = graph
            # layers = {f"add_to_thre11e-{(collection.name)}": {(f"add_to_three-{(collection.name)}", 0): (add_to_three, 20)}}
            # layers = {f"add_to_thre11e-{(collection.name)}": (add_to_three, 20)}
            # dependencies = {f"add_to_three-{(collection.name)}": set()}
            # graph = HighLevelGraph(layers, dependencies)
            # collection.dask = graph
            print("New Dask Graph")
            print(collection.dask)

    print("-----------------")

    return original_collections_to_dsk(collections, *args, **kwargs)

In [22]:
client.collections_to_dsk = modified_collections_to_dsk

In [23]:
def add_to_three(number):
    return number + 3

In [24]:
x = da.ones((1000, 1000), chunks=(100, 100))
y = x + 1
z = y.mean()
result = client.compute(z)

Inside myFunc. Time = 19:43:16
Functions already processed
mean_agg-aggregate-c50263a0ad47c330577b3058195ad928
Dask Graph Tasks
Task Name: mean_agg-aggregate-c50263a0ad47c330577b3058195ad928
Dask Graph
HighLevelGraph with 7 layers.
 0. ones_like-c53a97836143a00162e3470fef213e1e
 1. add-098526cf93d1ac0d3a8744f59d0814fb
 2. mean_chunk-4e5168d025885dfbdfd76df3f8083cda
 3. mean_combine-partial-0832826c732652f3243668d740a0d5f3
 4. mean_combine-partial-f33215f12dcb169d48ded3405ae5b4a2
 5. mean_combine-partial-f380b3499d120118c907e62d760350cc
 6. mean_agg-aggregate-c50263a0ad47c330577b3058195ad928

mean_agg-aggregate-c50263a0ad47c330577b3058195ad928 has already been processed
New Dask Graph
HighLevelGraph with 1 layers.
 0. add_to_three-mean_agg-aggregate-c50263a0ad47c330577b3058195ad928

-----------------


In [25]:
result

<Future: finished, type: str, key: finalize-3daf4f11543bcdbd0d181b8d6c929af3>

In [26]:
result.result()

'mean_agg-aggregate-c50263a0ad47c330577b3058195ad928'

### Example 3.3: Try with blockwise

In [5]:
from dask.distributed import Client
import dask
import dask.array as da
import numpy as np
from dask.highlevelgraph import HighLevelGraph
from datetime import datetime
from dask.array.core import Array

client = Client()

# Define some simple functions to use in the graph
def add(x, y):
    return x + y


def multiply(x, y):
    return x * y


# Create a Dask array from the HighLevelGraph
# Note: We need to specify the shape and chunks of the resulting array
dask_array = da.blockwise(
    lambda x, y: multiply(add(x, y), y),
    "i",
    np.array([1, 2, 3]),
    "i",
    np.array([4, 5, 6]),
    "i",
    dtype=np.int64,
)

# Use client.compute to compute the result
future = client.compute(dask_array)
result = future.result()

client.shutdown()

print("------------")
print(result)

/software/swift-hep/envs/dask_dirac/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 16985 instead
  warnings.warn(
2024-06-24 22:40:59,285 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:16327' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'finalize-99beb22034172632fdd161427fa0c2cc'} (stimulus_id='handle-worker-cleanup-1719265259.285501')


------------
[20 35 54]


In [6]:
from dask.distributed import Client
import dask
import dask.array as da
import numpy as np
from dask.highlevelgraph import HighLevelGraph
from datetime import datetime
from dask.array.core import Array


def modified_collections_to_dsk(collections, *args, **kwargs):
    c = datetime.now()
    print("Inside myFunc. Time = " + c.strftime("%H:%M:%S"))

    print("Functions already processed")

    print("Dask Graph Tasks")
    for collection in collections:
        print(f"Task Name: {collection.name}")
        print("Dask Graph")
        print(collection.dask)
        print("-----------------")
        print("Edditing Dask Graph")
        dask_array = da.blockwise(
            lambda x, y: multiply(add(x, y), y),
            "i",
            np.array([5, 6, 7]),
            "i",
            np.array([8, 9, 10]),
            "i",
            dtype=np.int64,
        )
        collection.dask = dask_array.dask
        collection.__dask_keys__ = dask_array.__dask_keys__

        print("New Dask Graph")
        print(collection.dask)

    print("-----------------")

    return original_collections_to_dsk(collections, *args, **kwargs)


client = Client()

original_collections_to_dsk = client.collections_to_dsk

client.collections_to_dsk = modified_collections_to_dsk

# Define some simple functions to use in the graph
def add(x, y):
    return x + y


def multiply(x, y):
    return x * y


# Create a Dask array from the HighLevelGraph
# Note: We need to specify the shape and chunks of the resulting array
dask_array = da.blockwise(
    lambda x, y: multiply(add(x, y), y),
    "i",
    np.array([1, 2, 3]),
    "i",
    np.array([4, 5, 6]),
    "i",
    dtype=np.int64,
)

# Use client.compute to compute the result
future = client.compute(dask_array)
result = future.result()

client.shutdown()

print("------------")
print(result)

/software/swift-hep/envs/dask_dirac/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 25436 instead
  warnings.warn(


Inside myFunc. Time = 22:41:44
Functions already processed
Dask Graph Tasks
Task Name: lambda-bd96b5ee7bd8b34908deaa71441f1d4a
Dask Graph
HighLevelGraph with 3 layers.
 0. array-93899268a02d75f85755ccb59b755ad1
 1. array-08b8023b2db8bebf73813d166e3f6b63
 2. lambda-bd96b5ee7bd8b34908deaa71441f1d4a

-----------------
Edditing Dask Graph
New Dask Graph
HighLevelGraph with 3 layers.
 0. array-71a7394e5f14de7dc72bb0b1b251ce84
 1. array-e3622cdb87acdb3eed57eec6c2ef9fb7
 2. lambda-19e9145bca12a9b712321a40e73e64bd

-----------------


2024-06-24 22:41:45,478 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:21393' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'finalize-a78071afc5b678a3bf5936b8a138ce3f'} (stimulus_id='handle-worker-cleanup-1719265305.478143')


------------
[104 135 170]


### Example 3.4: ones and blockwise

In this example, a HighLevelGraph containing two tasks is submitted to the client.
Prior to being sent to the scheduler, the HighLevelGraph is replaced with a HighLevelGraph defined by blockwise.


In [1]:
from dask.distributed import Client
import dask
import dask.array as da
import numpy as np
from dask.highlevelgraph import HighLevelGraph
from datetime import datetime
from dask.array.core import Array


def modified_collections_to_dsk(collections, *args, **kwargs):
    c = datetime.now()
    print("Inside myFunc. Time = " + c.strftime("%H:%M:%S"))

    print("Functions already processed")

    print("Dask Graph Tasks")
    for collection in collections:
        print(f"Task Name: {collection.name}")
        print("Dask Graph")
        print(collection.dask)
        print("-----------------")
        print("Edditing Dask Graph")
        dask_array = da.blockwise(
            lambda x, y: multiply(add(x, y), y),
            "i",
            np.array([5, 6, 7]),
            "i",
            np.array([8, 9, 10]),
            "i",
            dtype=np.int64,
        )
        collection.dask = dask_array.dask
        collection.__dask_keys__ = dask_array.__dask_keys__

        print("New Dask Graph")
        print(collection.dask)

    print("-----------------")

    return original_collections_to_dsk(collections, *args, **kwargs)


client = Client()

original_collections_to_dsk = client.collections_to_dsk

client.collections_to_dsk = modified_collections_to_dsk

# Define some simple functions to use in the graph
def add(x, y):
    return x + y


def multiply(x, y):
    return x * y


# Create a Dask array from the HighLevelGraph
# Note: We need to specify the shape and chunks of the resulting array
dask_array = da.ones((1000, 1000), chunks=(100,100)) + 1


# Use client.compute to compute the result
future = client.compute(dask_array)
result = future.result()

client.shutdown()

print("------------")
print(result)

Inside myFunc. Time = 16:05:15
Functions already processed
Dask Graph Tasks
Task Name: add-098526cf93d1ac0d3a8744f59d0814fb
Dask Graph
HighLevelGraph with 2 layers.
 0. ones_like-c53a97836143a00162e3470fef213e1e
 1. add-098526cf93d1ac0d3a8744f59d0814fb

-----------------
Edditing Dask Graph
New Dask Graph
HighLevelGraph with 3 layers.
 0. array-71a7394e5f14de7dc72bb0b1b251ce84
 1. array-e3622cdb87acdb3eed57eec6c2ef9fb7
 2. lambda-75f9fecc0d97ebf395a496c2575f03d0

-----------------


2024-06-25 16:05:15,426 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:13193' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'finalize-c0d5d498f2169d9602849de30cd6a0a0'} (stimulus_id='handle-worker-cleanup-1719327915.4264529')


------------
[104 135 170]


## Example 4: Hashing

### Example 4.1: Hashing

In this example, a hash is created using hashlib. 
dask.tokenize is not used as it is not repeatable in different environments.

Expected hash is `ec2f2283b6f02591655b83abe2be321d08b8b54983b0d9083da5eda81e837d490c838fdb882d8891161f832d30ff1cd8`

In [1]:
import dask.array as da
import hashlib

dask_array = da.ones((1000, 1000), chunks=(100,100)) + 20231
graph = dask_array.__dask_graph__()
print(graph)

HighLevelGraph with 2 layers.
 0. ones_like-c53a97836143a00162e3470fef213e1e
 1. add-34435cc4a153f02b208c5e30ee811e3d



In [2]:
total_graph_description = []
for i, (layer_name, layer) in enumerate(graph.layers.items()):
    print(f"Layer: {layer_name}")
    short_layer_name = layer_name[:layer_name.rfind('-')]
    print(f"short_layer_name: {short_layer_name}")
    print(f"len: {len(layer.items())}")
    # Lazy looping to get the args of one layer. 
    # Possible extention would get all hash of each layer
    for task_key, task in layer.items():
        function = task[0]
        args = task[1:]
    print(f"args: {args}")
    if i == 0:
        print(f'args: {args[0]}')
        layer_args = args[0]
        
    else:
        print(f'args: {args[1]}')
        layer_args = args[1]
    print(f'function: {function}')
    layer_description = {"function:": short_layer_name,
                         "layer_length": len(layer.items()),
                         "layer_args": layer_args}
    total_graph_description.append(layer_description)
    print('----------------------')

print('total graph description:')
print(total_graph_description)

Layer: ones_like-c53a97836143a00162e3470fef213e1e
short_layer_name: ones_like
len: 100
args: ((100, 100),)
args: (100, 100)
function: subgraph_callable-f646aaf1ae08e118dd013a3d16eb54ec
----------------------
Layer: add-34435cc4a153f02b208c5e30ee811e3d
short_layer_name: add
len: 100
args: (('ones_like-c53a97836143a00162e3470fef213e1e', 9, 9), 20231)
args: 20231
function: subgraph_callable-67aaf28825d396a0abf7512dfd79c838
----------------------
total graph description:
[{'function:': 'ones_like', 'layer_length': 100, 'layer_args': (100, 100)}, {'function:': 'add', 'layer_length': 100, 'layer_args': 20231}]


In [3]:
hashlib.sha3_384(str(total_graph_description).encode()).hexdigest()

'ec2f2283b6f02591655b83abe2be321d08b8b54983b0d9083da5eda81e837d490c838fdb882d8891161f832d30ff1cd8'

`ec2f2283b6f02591655b83abe2be321d08b8b54983b0d9083da5eda81e837d490c838fdb882d8891161f832d30ff1cd8`

is the expected result

In [4]:
dask_array.compute()

array([[20232., 20232., 20232., ..., 20232., 20232., 20232.],
       [20232., 20232., 20232., ..., 20232., 20232., 20232.],
       [20232., 20232., 20232., ..., 20232., 20232., 20232.],
       ...,
       [20232., 20232., 20232., ..., 20232., 20232., 20232.],
       [20232., 20232., 20232., ..., 20232., 20232., 20232.],
       [20232., 20232., 20232., ..., 20232., 20232., 20232.]])

### Example 4.2: Move the hash determination into collections_to_dsk

In [1]:
import dask.array as da
import hashlib
from datetime import datetime
from dask.distributed import Client
from pprint import pprint

In [2]:
def modified_collections_to_dsk(collections, *args, **kwargs):
    c = datetime.now()
    print("Inside myFunc. Time = " + c.strftime("%H:%M:%S"))

    print("Functions already processed")

    print("Dask Graph Tasks")
    for collection in collections:
        graph = collection.dask

        print(f"Task Name: {collection.name}")
        print("Dask Graph")
        print(graph)
        total_graph_description = []
        for i, (layer_name, layer) in enumerate(graph.layers.items()):
            short_layer_name = layer_name[:layer_name.rfind('-')]
            task_key, task = next(iter(layer.items()))
            function = task[0]
            task_args = task[1:]
            if i == 0:
                layer_args = task_args[0]
            else:
                layer_args = task_args[1]
    
            layer_description = {"function:": short_layer_name,
                                 "layer_length": len(layer.items()),
                                 "layer_args": layer_args}
            total_graph_description.append(layer_description)
        print('total_graph_description:')
        pprint(total_graph_description)
        graph_hash = hashlib.sha3_384(str(total_graph_description).encode()).hexdigest()
        print(f'hash: \033[93m {graph_hash} \033[0m')

    print('----------------------')

    return original_collections_to_dsk(collections, *args, **kwargs)

In [3]:
client = Client()

original_collections_to_dsk = client.collections_to_dsk

client.collections_to_dsk = modified_collections_to_dsk

dask_array = da.ones((1000, 1000), chunks=(100,100)) + 20231

# Use client.compute to compute the result
future = client.compute(dask_array)
result = future.result()

client.shutdown()

print("------------")
print(result)

Inside myFunc. Time = 20:57:57
Functions already processed
Dask Graph Tasks
Task Name: add-34435cc4a153f02b208c5e30ee811e3d
Dask Graph
HighLevelGraph with 2 layers.
 0. ones_like-c53a97836143a00162e3470fef213e1e
 1. add-34435cc4a153f02b208c5e30ee811e3d

total_graph_description:
[{'function:': 'ones_like', 'layer_args': (100, 100), 'layer_length': 100},
 {'function:': 'add', 'layer_args': 20231, 'layer_length': 100}]
hash:  ec2f2283b6f02591655b83abe2be321d08b8b54983b0d9083da5eda81e837d490c838fdb882d8891161f832d30ff1cd8 
----------------------


2024-07-02 20:57:58,396 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:5373' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'finalize-236d3b698353fe74f1e05c293988becb'} (stimulus_id='handle-worker-cleanup-1719950278.395871')


------------
[[20232. 20232. 20232. ... 20232. 20232. 20232.]
 [20232. 20232. 20232. ... 20232. 20232. 20232.]
 [20232. 20232. 20232. ... 20232. 20232. 20232.]
 ...
 [20232. 20232. 20232. ... 20232. 20232. 20232.]
 [20232. 20232. 20232. ... 20232. 20232. 20232.]
 [20232. 20232. 20232. ... 20232. 20232. 20232.]]


### Example 4.3: Change graph based on hash

In [1]:
import dask.array as da
import hashlib
from datetime import datetime
from dask.distributed import Client
from pprint import pprint
import numpy as np

In [2]:
def modified_collections_to_dsk(collections, *args, **kwargs):
    c = datetime.now()
    print("Inside myFunc. Time = " + c.strftime("%H:%M:%S"))

    print("Functions already processed")

    global existing_hashes

    print("Dask Graph Tasks")
    for collection in collections:
        graph = collection.dask

        print(f"Task Name: {collection.name}")
        print("Dask Graph")
        print(graph)
        total_graph_description = []
        for i, (layer_name, layer) in enumerate(graph.layers.items()):
            short_layer_name = layer_name[:layer_name.rfind('-')]
            task_key, task = next(iter(layer.items()))
            function = task[0]
            task_args = task[1:]
            if i == 0:
                layer_args = task_args[0]
            else:
                layer_args = task_args[1]
    
            layer_description = {"function:": short_layer_name,
                                 "layer_length": len(layer.items()),
                                 "layer_args": layer_args}
            total_graph_description.append(layer_description)
        print('total_graph_description:')
        pprint(total_graph_description)
        graph_hash = hashlib.sha3_384(str(total_graph_description).encode()).hexdigest()
        print(f'hash: \033[93m {graph_hash} \033[0m')
        if graph_hash in existing_hashes:
            print('\nGraph found in existing_hashes... replacing graph')
            dask_array = da.blockwise(
            lambda x, y: multiply(add(x, y), y),
            "i",
            np.array([5, 6, 7]),
            "i",
            np.array([8, 9, 10]),
            "i",
            dtype=np.int64,
            )
            collection.dask = dask_array.dask
            collection.__dask_keys__ = dask_array.__dask_keys__
            print('New Dask Graph')
            print(collection.dask)
    print('----------------------')

    return original_collections_to_dsk(collections, *args, **kwargs)

In [3]:
existing_hashes = ['ec2f2283b6f02591655b83abe2be321d08b8b54983b0d9083da5eda81e837d490c838fdb882d8891161f832d30ff1cd8']

def add(x, y):
    return x + y


def multiply(x, y):
    return x * y

In [4]:
client = Client()

original_collections_to_dsk = client.collections_to_dsk

client.collections_to_dsk = modified_collections_to_dsk

dask_array = da.ones((1000, 1000), chunks=(100,100)) + 20231

# Use client.compute to compute the result
future = client.compute(dask_array)
result = future.result()

client.shutdown()

print("------------")
print(result)

Inside myFunc. Time = 20:57:24
Functions already processed
Dask Graph Tasks
Task Name: add-34435cc4a153f02b208c5e30ee811e3d
Dask Graph
HighLevelGraph with 2 layers.
 0. ones_like-c53a97836143a00162e3470fef213e1e
 1. add-34435cc4a153f02b208c5e30ee811e3d

total_graph_description:
[{'function:': 'ones_like', 'layer_args': (100, 100), 'layer_length': 100},
 {'function:': 'add', 'layer_args': 20231, 'layer_length': 100}]
hash:  ec2f2283b6f02591655b83abe2be321d08b8b54983b0d9083da5eda81e837d490c838fdb882d8891161f832d30ff1cd8 

Graph found in existing_hashes... replacing graph
New Dask Graph
HighLevelGraph with 3 layers.
 0. array-71a7394e5f14de7dc72bb0b1b251ce84
 1. array-e3622cdb87acdb3eed57eec6c2ef9fb7
 2. lambda-83b0b2050c699e9e26592de20f8e625b

----------------------


2024-07-02 20:57:25,128 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:25830' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'finalize-236d3b698353fe74f1e05c293988becb'} (stimulus_id='handle-worker-cleanup-1719950245.1277432')


------------
[104 135 170]
